In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# ------------------------------
# 1. Load a small regression dataset
# ------------------------------
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import numpy as np

X, y = load_diabetes(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shapes:", X_train.shape, y_train.shape)

# ------------------------------
# 2. Build model function
# ------------------------------
def build_model(hp):
    model = keras.Sequential()
    
    # First layer needs input shape
    model.add(layers.Dense(
        units=hp.Int('units_0', min_value=32, max_value=256, step=32),
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    
    # Additional hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):  # keep it small
        model.add(layers.Dense(
            units=hp.Int('units_' + str(i), min_value=32, max_value=256, step=32),
            activation='relu'
        ))
    
    # Output layer for regression
    model.add(layers.Dense(1, activation='linear'))
    
    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )
    
    return model

# ------------------------------
# 3. Define tuner
# ------------------------------
tuner = kt.RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=3,              # just 3 trials to keep fast
    executions_per_trial=1,    # run once per trial
    directory='project',
    project_name='debug_demo'
)

# ------------------------------
# 4. Run tuner
# ------------------------------
tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32
)

# ------------------------------
# 5. Get best model
# ------------------------------
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Trial 3 Complete [00h 00m 02s]
val_mean_absolute_error: 45.00334548950195

Best val_mean_absolute_error So Far: 45.00334548950195
Total elapsed time: 00h 00m 07s


f:\aiml2\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,921 (304.38 KB)

 Trainable params: 77,921 (304.38 KB)

 Non-trainable params: 0 (0.00 B)